In [62]:
import os
import cv2

DATA_DIR = './data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

number_of_classes = 3
video_files = ['/content/A.mp4', '/content/B.mp4', '/content/L.mp4']
dataset_size = 100

# Function to collect data from a video file
def collect_data(video_path, class_index):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame if needed
        # frame = cv2.resize(frame, (desired_width, desired_height))

        # Save frame to the corresponding class directory
        frame_path = os.path.join(DATA_DIR, str(class_index), f'{frame_count}.jpg')
        cv2.imwrite(frame_path, frame)

        frame_count += 1

        # Stop collecting if dataset_size frames are collected
        if frame_count >= dataset_size:
            break

    cap.release()

# Loop over each class and collect data from respective video file
for class_index, video_file in enumerate(video_files):
    class_dir = os.path.join(DATA_DIR, str(class_index))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    print(f'Collecting data for class {class_index}')
    collect_data(video_file, class_index)

print('Data collection completed.')


Data collection completed.


In [63]:
import os
import cv2

DATA_DIR = './data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

number_of_classes = 3
video_files = ['/content/A.mp4', '/content/B.mp4', '/content/L.mp4']
dataset_size = 100

# Function to collect data from a video file
def collect_data(video_path, class_index):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame if needed
        # frame = cv2.resize(frame, (desired_width, desired_height))

        # Save frame to the corresponding class directory
        class_dir = os.path.join(DATA_DIR, str(class_index))
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

        frame_path = os.path.join(class_dir, f'{frame_count}.jpg')
        cv2.imwrite(frame_path, frame)

        frame_count += 1

        # Stop collecting if dataset_size frames are collected
        if frame_count >= dataset_size:
            break

    cap.release()

# Loop over each class and collect data from respective video file
for class_index, video_file in enumerate(video_files):
    print(f'Collecting data for class {class_index} from {video_file}')
    collect_data(video_file, class_index)

print('Data collection completed.')


Data collection completed.


For example, here we download and display a PNG image of the Colab logo:

In [64]:
import os
import pickle

import mediapipe as mp
import cv2
import matplotlib.pyplot as plt


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = './data'

data = []
labels = []
for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []

        x_ = []
        y_ = []

        img_full_path = os.path.join(DATA_DIR, dir_, img_path)
        # Check if the file is a valid image file, skip if it's a directory or checkpoint file
        if not (os.path.isfile(img_full_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg'))):
            continue

        img = cv2.imread(img_full_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            data.append(data_aux)
            labels.append(dir_)

f = open('data.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [65]:
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


data_dict = pickle.load(open('./data.pickle', 'rb'))

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

model = RandomForestClassifier()

model.fit(x_train, y_train)

y_predict = model.predict(x_test)

score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))

f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

100.0% of samples were classified correctly !


In [68]:
import os
import cv2
import mediapipe as mp
import numpy as np
from google.colab import files

# Install mediapipe if not already installed
!pip install mediapipe

# Load your model from pickle file
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

def get_sentence(character):
    if character == 'A':
        return "What is diabetics"
    elif character == 'B':
        return "What role do medications play in the treatment of diabetes?"
    elif character == 'L':
        return "What are the three main types of diabetes?"
    else:
        return "Unknown character"

# Function to process a video file
def process_video(video_path):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)
    labels_dict = {0: 'A', 1: 'B', 2: 'L'}

    cap = cv2.VideoCapture(video_path)

    # Get video details
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    output_video_path = 'output_video.avi'
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    questions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        H, W, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(frame_rgb)
        if results.multi_hand_landmarks:
            data_aux = []
            x_ = []
            y_ = []

            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10
            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            prediction = model.predict([np.asarray(data_aux)])
            predicted_character = labels_dict[int(prediction[0])]

            sentence = get_sentence(predicted_character)
            questions.append(sentence)  # Append the question to array

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)

        out.write(frame)

    cap.release()
    out.release()

    return output_video_path, questions

# Process the uploaded video 'new.mp4'
video_path = '/content/WIN_20240620_16_56_45_Pro.mp4'  # Replace with the path to your uploaded video 'new.mp4'
output_video_path, questions = process_video(video_path)

# Download the processed video
files.download(output_video_path)

print("Questions:")
for q in questions:
    print(q)


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.10/dist-packages/google/protobuf/sy

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Questions:
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What role do medications play in the treatment of diabetes?
What are the three main types of diabetes?
What are the three main types of diabetes?
What are the three main types of diabetes?
What are the three main types of diabetes?
What are the three main types of diabetes?
What are the three main types of diabetes?
What are th

In [69]:
print(questions)

['What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What role do medications play in the treatment of diabetes?', 'What are the three main types of diabetes?', 'What are the three main types of diabetes?', 'What are the three main types of diabetes?', 'What are the three main types of diabetes?', 'What are the three main types of diabetes?', 'What are the

In [70]:
unique_questions = []

for q in questions:
    if q not in unique_questions:
        unique_questions.append(q)

print("Unique Questions:")
for q in unique_questions:
    print(q)

Unique Questions:
What role do medications play in the treatment of diabetes?
What are the three main types of diabetes?
What is diabetics


In [71]:

!pip3 install fse
!pip3 install transformers
!pip3 install torch
!git clone -l -s https://github.com/harpreetvirkk/NLP-QuestionAnswerSystem rep
%cd rep
!ls dataset


Cloning into 'rep'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 185 (delta 28), reused 161 (delta 18), pack-reused 0
Receiving objects: 100% (185/185), 2.42 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/rep/rep
S08_question_answer_pairs.txt  S10_question_answer_pairs.txt  text_data_toc.csv
S09_question_answer_pairs.txt  text_data


In [72]:
import math
import os
import re
import fse
import gensim
import gensim.downloader as api
import nltk
import numpy as np
import pandas as pd
from fse import SplitIndexedList
from fse.models import uSIF
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('stopwords')
import logging
import re
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from nltk.tokenize import word_tokenize
from transformers import BertForQuestionAnswering, BertTokenizer
pd.set_option('display.max_colwidth', None) # Use None to display the full width

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
# Reading in the dataset
df1 = pd.read_csv('./dataset/S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('./dataset/S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('./dataset/S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')
frames = [df1, df2, df3]
df = pd.concat(frames)

In [74]:
df3.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemistry.,easy,easy,S10_set4_a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,S10_set4_a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely operated pistol?,Alessandro Volta did invent the remotely operated pistol.,easy,easy,S10_set4_a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely operated pistol?,Yes,easy,easy,S10_set4_a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,S10_set4_a10


In [75]:
def getArticleText(file):
  fpath = '/content/rep/dataset/text_data/'+file+'.txt.clean'
  try:
    f = open(fpath, 'r')
    text = f.read()
  except UnicodeDecodeError:
    f = open(fpath, 'r', encoding = 'ISO-8859-1')
    text = f.read()
  return text

In [76]:

df = df.dropna(subset=['ArticleFile'])
df = df.dropna(subset=['Answer'])
df['ArticleText'] = df['ArticleFile'].apply(lambda x: getArticleText(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: re.sub(r'(\n)+', '. ', x))
df = df.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis='columns')
df

ArticleTitle  \
0     Abraham_Lincoln   
1     Abraham_Lincoln   
2     Abraham_Lincoln   
3     Abraham_Lincoln   
4     Abraham_Lincoln   
...               ...   
1452            Zebra   
1454            Zebra   
1455            Zebra   
1456            Zebra   
1457            Zebra   

                                                               Question  \
0     Was Abraham Lincoln the sixteenth President of the United States?   
1     Was Abraham Lincoln the sixteenth President of the United States?   
2                    Did Lincoln sign the National Banking Act of 1863?   
3                    Did Lincoln sign the National Banking Act of 1863?   
4                                      Did his mother die of pneumonia?   
...                                                                 ...   
1452                          What areas do the Grevy's Zebras inhabit?   
1454               Which species of zebra is known as the common zebra?   
1455               Which species of zebra is known as the common zebra?   
1456                                     At what age can a zebra breed?   
1457                                     At what age can a zebra breed?   

                                                     Answer  \
0                                                       yes   
1                                                      Yes.   
2                                                       yes   
3                                                      Yes.   
4                                                        no   
...                                                     ...   
1452    semi-arid grasslands of Ethiopia and northern Kenya   
1454  Plains Zebra (Equus quagga, formerly Equus burchelli)   
1455                                           Plains Zebra   
1456                                            five or six   
1457                                                 5 or 6   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [77]:
# stop_words = set(stopwords.words('english'))
def cleanQuestion(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanAnswer(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanText(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s\.\?]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

In [78]:
df['Question'] = df['Question'].apply(lambda x: cleanQuestion(x))
df['Answer'] = df['Answer'].apply(lambda x: cleanAnswer(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: cleanText(x))
df

ArticleTitle  \
0     Abraham_Lincoln   
1     Abraham_Lincoln   
2     Abraham_Lincoln   
3     Abraham_Lincoln   
4     Abraham_Lincoln   
...               ...   
1452            Zebra   
1454            Zebra   
1455            Zebra   
1456            Zebra   
1457            Zebra   

                                                              Question  \
0     was abraham lincoln the sixteenth president of the united states   
1     was abraham lincoln the sixteenth president of the united states   
2                    did lincoln sign the national banking act of 1863   
3                    did lincoln sign the national banking act of 1863   
4                                      did his mother die of pneumonia   
...                                                                ...   
1452                           what areas do the grevys zebras inhabit   
1454               which species of zebra is known as the common zebra   
1455               which species of zebra is known as the common zebra   
1456                                     at what age can a zebra breed   
1457                                     at what age can a zebra breed   

                                                  Answer  \
0                                                    yes   
1                                                    yes   
2                                                    yes   
3                                                    yes   
4                                                     no   
...                                                  ...   
1452  semiarid grasslands of ethiopia and northern kenya   
1454  plains zebra equus quagga formerly equus burchelli   
1455                                        plains zebra   
1456                                         five or six   
1457                                              5 or 6   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [79]:
# Preparing data for training
allQuestion = '. '.join(list(df['Question']))
allAnswer = '. '.join(list(df['Answer']))
allContext = '. '.join(list(df['ArticleText']))
text = allQuestion + allAnswer + allContext
s = SplitIndexedList(text.split('.'))

In [80]:
use_cuda = True
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:

def getGraph(start_scores, end_scores, tokens):

  # x-axis: unique tokens
  # y-axis: start word score for each token

  # Pull the scores from Tensors and convert to 1-D np arrays
  start_scores = start_scores.detach().numpy().flatten()
  end_scores = end_scores.detach().numpy().flatten()

  # We add unique token index to each token
  token_label = list()
  for (i, t) in enumerate(tokens):
      token_label.append('{:} - {:>2}'.format(t, i))

  # Plot Graph
  sns.set(style='darkgrid')
  plt.rcParams["figure.figsize"] = (24,8)
  ax = sns.barplot(x=token_labels, y=s_scores, ci=None)
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
  ax.grid(True)
  plt.title('Start Word Scores for Each unique Token in Context')
  plt.show()

In [82]:
from fse.models import uSIF
glove = api.load("glove-wiki-gigaword-100")
model_sif = uSIF(glove, workers=2, lang_freq="en")


In [83]:
model_sif.train(SplitIndexedList(list(df['ArticleText'])))


(3418, 14826914)

In [84]:

def get_split(text1):

  #Reference: https://medium.com/@armandj.olivares/using-bert-for-classifying-documents-with-long-texts-5c3e7b04573d

  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else:
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:250]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 250]
      l_total.append(" ".join(l_parcial))
  return l_total

In [85]:
def getAnswerBert(question, context):

  # print('Query Context has {} tokens.'.format(len(tokenizer.encode(context))))

  context_list = get_split(context)
  print(context_list)

  ans = []

  for c in context_list:

    encoding = tokenizer.encode_plus(text=question,text_pair=c)

    inputs = encoding['input_ids']  #Token embeddings
    token_type_id = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

    output = model_bert(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([token_type_id]))
    start_index = torch.argmax(output.start_logits)
    end_index = torch.argmax(output.end_logits)

    answer = ' '.join(tokens[start_index:end_index+1])

    ans.append(answer)
  print('Question: ', question)

  potentials = []
  for i in ans:
    if ('SEP' not in i) and ('CLS' not in i):
      potentials.append(re.sub('(#)+', '', i))

  # You need to define or import the getBestAnswer function
  # For example, if you have a simple logic to select the best answer, you could do:
  answer = max(potentials, key=len)  # Select the longest potential answer

  # print('Potential Answers: \n')
  # print(answer.head())
  return answer

In [86]:
print(getAnswerBert(df['Question'].iloc[171], df['ArticleText'].iloc[171]))
print('Actual Answer: ', df['Answer'].iloc[171])

['calvin coolidge. . john calvin coolidge jr. july 4 1872 january 5 1933 more commonly known as calvin coolidge was the thirtieth president of the united states 19231929. a lawyer from vermont coolidge worked his way up the ladder of massachusetts state politics eventually becoming governor of that state. his actions during the boston police strike of 1919 thrust him into the national spotlight. soon after he was elected as the twentyninth vice president in 1920 and succeeded to the presidency upon the death of warren g. harding. elected in his own right in 1924 he gained a reputation as a smallgovernment conservative.. in many ways coolidges style of governance was a throwback to the passive presidency of the nineteenth century. sobel 14 he restored public confidence in the white house after the scandals of his predecessors administration and left office with considerable popularity. mccoy 420421 greenberg 4953 as his biographer later put it he embodied the spirit and hopes of the mid

In [87]:
print(getAnswerBert(df['Question'].iloc[255], df['ArticleText'].iloc[255]))
print('Actual Answer: ', df['Answer'].iloc[255])

['duck. duck is the common name for a number of species in the anatidae family of birds. the ducks are divided between several subfamilies listed in full in the anatidae article. ducks are mostly aquatic birds mostly smaller than their relatives the swans and geese and may be found in both fresh water and sea water.. most ducks have a wide flat beak adapted for dredging. they exploit a variety of food sources such as grasses aquatic plants fish insects small amphibians photo of a duck eating a frog worms and small molluscs.. ducks are sometimes confused with several types of unrelated water birds with similar forms such as loons or divers grebes gallinules and coots.. many species of duck are temporarily flightless while moulting they seek out protected habitat with good food supplies during this period. this moult typically precedes migration.. some duck species mainly those breeding in the temperate and arctic northern hemisphere are migratory those in the tropics however are general

In [88]:
def ask_question(question, context):
    answer = getAnswerBert(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    return answer

# Example usage
# Provide your own question and context
question = unique_questions[2]
context = '''Diabetes mellitus, commonly referred to as diabetes, is a chronic metabolic disorder characterized by elevated levels of blood glucose (blood sugar), which over time leads to serious damage to various organs and tissues of the body. There are three main types of diabetes: Type 1 diabetes, Type 2 diabetes, and gestational diabetes. Each type has distinct etiologies, but all involve the body's inability to effectively regulate blood sugar levels.

Type 1 diabetes, also known as juvenile diabetes, is an autoimmune condition in which the immune system mistakenly attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production, necessitating lifelong insulin therapy for those affected. It usually manifests in childhood or adolescence, although it can occur at any age. The exact cause of Type 1 diabetes is unknown, but it is believed to involve a combination of genetic predisposition and environmental factors, such as viral infections.

Type 2 diabetes, the most common form, typically develops in adults over the age of 45, though it is increasingly being diagnosed in younger populations, including children, adolescents, and young adults. It is primarily associated with insulin resistance, a condition in which the body's cells do not respond properly to insulin, combined with an eventual decline in insulin production. Risk factors for Type 2 diabetes include obesity, physical inactivity, poor diet, and a family history of diabetes. Unlike Type 1, Type 2 diabetes can often be managed with lifestyle changes such as improved diet, increased physical activity, and weight loss, although medications or insulin therapy may also be necessary.

Gestational diabetes occurs during pregnancy and is characterized by high blood sugar levels that were not present before pregnancy. This type of diabetes typically resolves after childbirth, but it increases the risk of developing Type 2 diabetes later in life for both the mother and child. Gestational diabetes requires careful monitoring and management to minimize complications for both mother and baby.

Regardless of the type, uncontrolled diabetes can lead to a multitude of complications. Chronic high blood sugar levels can cause damage to blood vessels, leading to cardiovascular diseases such as heart attack and stroke, as well as microvascular complications affecting the eyes (retinopathy), kidneys (nephropathy), and nerves (neuropathy). Diabetic neuropathy can result in pain, tingling, and loss of sensation in the extremities, which, combined with poor circulation, increases the risk of foot ulcers and infections, potentially leading to amputations.

Effective management of diabetes involves a multifaceted approach. Regular monitoring of blood glucose levels is crucial for making informed treatment decisions. A balanced diet, rich in fruits, vegetables, whole grains, and lean proteins, helps maintain blood sugar levels within the target range. Physical activity enhances insulin sensitivity and aids in weight management. Medications, whether oral hypoglycemic agents or insulin, are often necessary to achieve optimal glucose control. Education and support from healthcare providers enable individuals with diabetes to understand their condition and take proactive steps in managing it.

Living with diabetes requires significant lifestyle adjustments and ongoing medical care. However, with proper management, individuals with diabetes can lead healthy and fulfilling lives. Advances in diabetes research continue to improve treatment options and bring hope for a cure in the future. Nonetheless, the increasing prevalence of diabetes worldwide underscores the importance of preventive measures, early detection, and comprehensive care to mitigate the impact of this pervasive disease.'''
# Get the answer
ask_question(question, context)

["Diabetes mellitus, commonly referred to as diabetes, is a chronic metabolic disorder characterized by elevated levels of blood glucose (blood sugar), which over time leads to serious damage to various organs and tissues of the body. There are three main types of diabetes: Type 1 diabetes, Type 2 diabetes, and gestational diabetes. Each type has distinct etiologies, but all involve the body's inability to effectively regulate blood sugar levels. Type 1 diabetes, also known as juvenile diabetes, is an autoimmune condition in which the immune system mistakenly attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production, necessitating lifelong insulin therapy for those affected. It usually manifests in childhood or adolescence, although it can occur at any age. The exact cause of Type 1 diabetes is unknown, but it is believed to involve a combination of genetic predisposition and environmental factors, such as viral infections. T

'diabetes mel lit us , commonly referred to as diabetes , is a chronic metabolic disorder characterized by elevated levels of blood glucose ( blood sugar )'

In [89]:
question = unique_questions[0]
ask_question(question, context)

["Diabetes mellitus, commonly referred to as diabetes, is a chronic metabolic disorder characterized by elevated levels of blood glucose (blood sugar), which over time leads to serious damage to various organs and tissues of the body. There are three main types of diabetes: Type 1 diabetes, Type 2 diabetes, and gestational diabetes. Each type has distinct etiologies, but all involve the body's inability to effectively regulate blood sugar levels. Type 1 diabetes, also known as juvenile diabetes, is an autoimmune condition in which the immune system mistakenly attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production, necessitating lifelong insulin therapy for those affected. It usually manifests in childhood or adolescence, although it can occur at any age. The exact cause of Type 1 diabetes is unknown, but it is believed to involve a combination of genetic predisposition and environmental factors, such as viral infections. T

'necessary to achieve optimal glucose control'

In [90]:
question = unique_questions[1]
ask_question(question, context)

["Diabetes mellitus, commonly referred to as diabetes, is a chronic metabolic disorder characterized by elevated levels of blood glucose (blood sugar), which over time leads to serious damage to various organs and tissues of the body. There are three main types of diabetes: Type 1 diabetes, Type 2 diabetes, and gestational diabetes. Each type has distinct etiologies, but all involve the body's inability to effectively regulate blood sugar levels. Type 1 diabetes, also known as juvenile diabetes, is an autoimmune condition in which the immune system mistakenly attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production, necessitating lifelong insulin therapy for those affected. It usually manifests in childhood or adolescence, although it can occur at any age. The exact cause of Type 1 diabetes is unknown, but it is believed to involve a combination of genetic predisposition and environmental factors, such as viral infections. T

'type 1 diabetes , type 2 diabetes , and ge station al diabetes'